In [1]:
# %pip install pyreadr
# %pip install catboost
# %pip install lckr-jupyterlab-variableinspector


In [2]:
import sklearn 
import os, sys
import pyreadr
import pandas as pd
import catboost
from IPython.display import clear_output


In [3]:
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/cv_surgery'
exec(open(work_dir+'/code/Python/snippets/helpful_functions.py').read())


In [4]:
import multiprocessing as mp
mp.cpu_count()

20

In [5]:
outcome = 'sts_14d'

In [6]:
load_dir = work_dir+'/csv_files/pre_anat_intra'
feature_space = pd.read_csv(load_dir+'/feature_space_preop_anat_intra_w_interaction_terms.csv') 

In [7]:
feature_space = feature_space.rename(columns={"feature_space.concatid": "concatid"}).drop(['Unnamed: 0'], axis =1)
feature_space.head()

,concatid,proctype,gender,ethnicity,raceasian,raceblack,racecaucasian,raceother,age,heightcm,...,nc_stern.cplegia_ant,nc_stern.cplegia_ret,nc_stern.frepl,nc_stern.inc_revasc,cplegia_ant.cplegia_ret,cplegia_ant.frepl,cplegia_ant.inc_revasc,cplegia_ret.frepl,cplegia_ret.inc_revasc,frepl.inc_revasc
0,10020V313492,2,2,2,2,2,1,2,54.0,155.0,...,2,2,4,4,1,2,2,2,2,4
1,10022V3117559,2,1,2,2,2,1,2,74.0,174.0,...,2,4,4,4,2,2,2,4,4,4
2,10022V315180,6,2,2,2,2,1,2,66.0,157.0,...,2,2,4,2,1,2,1,2,1,2
3,10031V313571,1,2,2,2,2,1,2,79.0,155.5,...,2,2,4,4,1,2,2,2,2,4
4,10047V3420470,1,1,2,2,2,1,2,61.0,170.0,...,4,4,4,4,4,4,4,4,4,4


In [8]:
#label_space = pd.read_csv(smb2.open('//'+work_dir+'/label_space_sts.csv'))
label_space = pd.read_csv(load_dir+'/label_space.csv')

In [9]:
label_space_no_na = label_space.fillna(2)

In [10]:
label_df = label_space_no_na[['concatid', outcome]].drop_duplicates(subset='concatid', keep="last")

In [11]:
label_df.head()

,concatid,sts_14d
0,10020V313492,2
1,10022V3117559,1
2,10022V315180,2
3,10031V313571,2
4,10047V3420470,2


In [12]:
data = pd.merge(label_df, feature_space.drop_duplicates(subset='concatid', keep="last")
                , how='left', on='concatid', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [13]:
data_full = data
data = data_full.drop(['concatid'], axis = 1)

In [14]:
data

,sts_14d,proctype,gender,ethnicity,raceasian,raceblack,racecaucasian,raceother,age,heightcm,...,nc_stern.cplegia_ant,nc_stern.cplegia_ret,nc_stern.frepl,nc_stern.inc_revasc,cplegia_ant.cplegia_ret,cplegia_ant.frepl,cplegia_ant.inc_revasc,cplegia_ret.frepl,cplegia_ret.inc_revasc,frepl.inc_revasc
0,2,2,2,2,2,2,1,2,54.0,155.0,...,2,2,4,4,1,2,2,2,2,4
1,1,2,1,2,2,2,1,2,74.0,174.0,...,2,4,4,4,2,2,2,4,4,4
2,2,6,2,2,2,2,1,2,66.0,157.0,...,2,2,4,2,1,2,1,2,1,2
3,2,1,2,2,2,2,1,2,79.0,155.5,...,2,2,4,4,1,2,2,2,2,4
4,2,1,1,2,2,2,1,2,61.0,170.0,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34556,1,7,1,2,2,2,1,2,81.0,178.1,...,2,4,4,4,2,2,2,4,4,4
34557,2,2,1,2,2,2,1,2,66.0,191.0,...,2,4,4,4,2,2,2,4,4,4
34558,2,2,1,2,2,2,1,2,82.0,188.0,...,4,4,4,4,4,4,4,4,4,4
34559,2,2,1,2,2,2,1,2,55.0,185.0,...,2,4,4,4,2,2,2,4,4,4


In [15]:
data_full.shape

(34561, 15756)

In [16]:
name_list = list(feature_space)


In [17]:
data.rename(columns={outcome: 'label'}, inplace=True)

In [18]:
# change label: 2 (alive) to 0 (class 0)
data.loc[data.label ==2, 'label'] = 0

In [19]:
data['label'].value_counts()


0    28961
1     5600
Name: label, dtype: int64

In [20]:
# Split train set/test set in stratified fashion, then downsample training set to balance outcome class:
from sklearn import model_selection
outercv = sklearn.model_selection.StratifiedKFold(n_splits=10,shuffle=True,random_state= 1)
X = data.drop(['label'],axis =1).values
y = data[['label']].values

for train_index, test_index in outercv.split(X, y):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

train_df = data.iloc[train_index]
test_df = data.iloc[test_index]
train_df_majority = train_df[train_df['label']==0]
train_df_minority = train_df[train_df['label']==1]

# Downsampling:
from sklearn.utils import resample
train_df_majority_downsampled = resample(train_df_majority, 
                                 replace=False,     # sample without replacement
                                 n_samples=2*train_df_minority.shape[0],    # to match minority class
                                 random_state=1)

df_downsampled = pd.concat([train_df_majority_downsampled, train_df_minority])

In [21]:
# y_test: 1 = dead, 2 = live. Convert live to class 0:

import numpy as np
np.array(np.unique(y_test, return_counts=True)).T
y_test[y_test == 2] = 0
y_train[y_train == 2] = 0
np.array(np.unique(y_test, return_counts=True)).T


array([[   0, 2896],
       [   1,  560]])

In [22]:
# Split the training set further to cv_train-validation set, using stratified k-fold cv: 10-fold x 5 times
# train and calibrate models
from sklearn import model_selection
outercv = sklearn.model_selection.RepeatedStratifiedKFold(n_splits=5, n_repeats = 2,random_state= 1)

# initialize:
cv_performance_df = pd.DataFrame()
cv_performance_isotonic_df = pd.DataFrame()
cv_performance_sigmoid_df = pd.DataFrame()
training_bs_testset_pred_df = data_full[['concatid',outcome]].loc[test_index,:].rename(columns={outcome: 'label'})
training_bs_testset_pred_calibrated_isotonic_df = training_bs_testset_pred_df.copy(deep = True)
training_bs_testset_pred_calibrated_sigmoid_df = training_bs_testset_pred_df.copy(deep = True)

fold = 1
for cv_train_index, cv_test_index in outercv.split(X_train, y_train):
    print("Fold "+ str(fold) + "....")
    fold = fold + 1
    
    # print("TRAIN:", train_index, "TEST:", test_index)
    cv_X_train, cv_X_test = X_train[cv_train_index], X_train[cv_test_index]
    cv_y_train, cv_y_test = y_train[cv_train_index], y_train[cv_test_index]
    
    cv_train_df = data.iloc[cv_train_index]
    cv_test_df = data.iloc[cv_test_index]
    cv_train_df_majority = cv_train_df[cv_train_df['label']==0]
    cv_train_df_minority = cv_train_df[cv_train_df['label']==1]

    # Downsampling:
    from sklearn.utils import resample
    cv_train_df_majority_downsampled = resample(cv_train_df_majority, 
                                     replace=False,     # sample without replacement
                                     n_samples=2*cv_train_df_minority.shape[0],    # to match minority class
                                     random_state=1)

    # Combine downsampled majority class with minority class
    cv_df_downsampled = pd.concat([cv_train_df_majority_downsampled, cv_train_df_minority])
    cv_X_train_downsampled = cv_df_downsampled.drop(['label'],axis =1)
    cv_y_train_downsampled = cv_df_downsampled['label'].ravel()
    
    
    # Train model:
    from catboost import CatBoostClassifier
    cv_cb_downsampled =  CatBoostClassifier(random_state = 1)
    cv_cb_downsampled.fit(cv_X_train_downsampled, cv_y_train_downsampled, verbose = False)
    clear_output()


    
    # Evaluate model on cv_test_set (validation set):
    cv_y_test_predict_proba_cb_downsampled = cv_cb_downsampled.predict_proba(cv_X_test)[:, 1]
    performance_metrics = performance_wo_figure(cv_y_test, cv_y_test_predict_proba_cb_downsampled, thres = 0.5)
    cv_performance_df = cv_performance_df.append(performance_metrics, ignore_index=True)
    
    
    # Pred prob of model on holdout testset:
    training_bs_testset_pred_df['time_'+str(fold-1)] = cv_cb_downsampled.predict_proba(X_test)[:, 1]
    
    
    
    
    # calibrate model, record performance metric validation set, and pred prob on holdout testset:
    cv_cb_downsampled_isotonic_on_cv = calibrate_model(cv_cb_downsampled, cv_X_train, cv_y_train.ravel(), method = 'isotonic')
    clear_output()
    training_bs_testset_pred_calibrated_isotonic_df['time_'+str(fold-1)] = cv_cb_downsampled_isotonic_on_cv.predict_proba(X_test)[:, 1]

    cv_y_test_predict_proba_cb_downsampled_isotonic = cv_cb_downsampled_isotonic_on_cv.predict_proba(cv_X_test)[:, 1]
    clear_output()
    performance_metrics_isotonic = performance_wo_figure(cv_y_test, cv_y_test_predict_proba_cb_downsampled_isotonic, thres = 0.5)
    cv_performance_isotonic_df = cv_performance_isotonic_df.append(performance_metrics_isotonic, ignore_index=True)

    
    cv_cb_downsampled_sigmoid_on_cv = calibrate_model(cv_cb_downsampled, cv_X_train, cv_y_train.ravel(), method = 'sigmoid')
    clear_output()

    training_bs_testset_pred_calibrated_sigmoid_df['time_'+str(fold-1)] = cv_cb_downsampled_sigmoid_on_cv.predict_proba(X_test)[:, 1]
    
    cv_y_test_predict_proba_cb_downsampled_sigmoid = cv_cb_downsampled_sigmoid_on_cv.predict_proba(cv_X_test)[:, 1]
    performance_metrics_sigmoid = performance_wo_figure(cv_y_test, cv_y_test_predict_proba_cb_downsampled_sigmoid, thres = 0.5)
    cv_performance_sigmoid_df = cv_performance_sigmoid_df.append(performance_metrics_sigmoid, ignore_index=True)
    
    clear_output()


In [23]:
training_bs_testset_pred_df.to_csv(load_dir+'/'+'patient_specific_pred_prob_testset_' + outcome +'.csv', index=False)
training_bs_testset_pred_calibrated_isotonic_df.to_csv(load_dir+'/'+'patient_specific_pred_prob_testset_isotonic_on_cv_' + outcome +'.csv', index=False)
training_bs_testset_pred_calibrated_sigmoid_df.to_csv(load_dir+'/'+'patient_specific_pred_prob_testset_sigmoid_on_cv_' + outcome +'.csv', index=False)


In [24]:
cv_performance_df.to_csv(load_dir+'/'+'cv_performance_df_' + outcome +'.csv', index=False)
cv_performance_df.describe(percentiles = [.025, .5, .975]).round(decimals = 3)


,ARC,AUROC,Brier,F1,accuracy,precision,recall,sens,spec
count,10.000,10.000,10.000,10.000,10.000,10.000,10.000,10.000,10.000
mean,0.720,0.916,0.089,0.677,0.884,0.616,0.752,0.752,0.909
std,0.019,0.007,0.003,0.009,0.004,0.012,0.012,0.012,0.005
min,0.679,0.900,0.087,0.655,0.874,0.590,0.737,0.737,0.901
2.5%,0.686,0.903,0.087,0.659,0.876,0.594,0.738,0.738,0.902
50%,0.723,0.917,0.088,0.679,0.885,0.617,0.751,0.751,0.910
97.5%,0.749,0.925,0.095,0.686,0.888,0.632,0.774,0.774,0.916
max,0.752,0.925,0.096,0.687,0.889,0.634,0.778,0.778,0.917


In [25]:
# print out performance:
print_cv_performance = pd.DataFrame(cv_performance_df[['AUROC', 'accuracy', 'ARC', 'Brier', 'F1','precision','recall', 'sens', 'spec']].apply(lambda x: report_metric(x), axis = 0)).transpose()

print_cv_performance.to_csv(load_dir+'/'+'print_cv_performance_' + outcome +'.csv', index=False)
print_cv_performance

,AUROC,accuracy,ARC,Brier,F1,precision,recall,sens,spec
0,"0.917 (0.903, 0.925)","0.885 (0.876, 0.888)","0.723 (0.686, 0.749)","0.088 (0.087, 0.095)","0.679 (0.659, 0.686)","0.617 (0.594, 0.632)","0.751 (0.738, 0.774)","0.751 (0.738, 0.774)","0.91 (0.902, 0.916)"


In [26]:
cv_performance_isotonic_df.to_csv(load_dir+'/'+'cv_performance_isotonic_df_' + outcome +'.csv', index=False)
print_cv_performance_isotonic = pd.DataFrame(cv_performance_isotonic_df[['AUROC', 'accuracy', 'ARC', 'Brier', 'F1','precision','recall', 'sens', 'spec']].apply(lambda x: report_metric(x), axis = 0)).transpose()
print_cv_performance_isotonic.to_csv(load_dir+'/'+'print_cv_performance_isotonic_' + outcome +'.csv', index=False)
print_cv_performance_isotonic

,AUROC,accuracy,ARC,Brier,F1,precision,recall,sens,spec
0,"0.856 (0.837, 0.862)","0.867 (0.86, 0.872)","0.58 (0.543, 0.607)","0.097 (0.094, 0.101)","0.42 (0.389, 0.449)","0.725 (0.659, 0.749)","0.299 (0.269, 0.324)","0.299 (0.269, 0.324)","0.977 (0.971, 0.981)"


In [27]:
cv_performance_sigmoid_df.to_csv(load_dir+'/'+'cv_performance_sigmoid_df_' + outcome +'.csv', index=False)
print_cv_performance_sigmoid = pd.DataFrame(cv_performance_sigmoid_df[['AUROC', 'accuracy', 'ARC', 'Brier', 'F1','precision','recall', 'sens', 'spec']].apply(lambda x: report_metric(x), axis = 0)).transpose()
print_cv_performance_sigmoid.to_csv(load_dir+'/'+'print_cv_performance_sigmoid_' + outcome +'.csv', index=False)
print_cv_performance_sigmoid

,AUROC,accuracy,ARC,Brier,F1,precision,recall,sens,spec
0,"0.856 (0.838, 0.863)","0.869 (0.861, 0.874)","0.579 (0.543, 0.606)","0.098 (0.095, 0.103)","0.442 (0.411, 0.478)","0.711 (0.644, 0.728)","0.321 (0.292, 0.356)","0.321 (0.292, 0.356)","0.974 (0.967, 0.977)"


In [28]:
# Save models:
import pickle
pickle.dump(cv_cb_downsampled, open(work_dir+'/models/pre_anat_intra/'+'model_' +outcome + '_cv_cb_downsampled.sav', 'wb'))
pickle.dump(cv_cb_downsampled_isotonic_on_cv, open(work_dir+'/models/pre_anat_intra/'+'model_' +outcome + '_cv_cb_downsampled_isotonic_calibration_on_cv.sav', 'wb'))
pickle.dump(cv_cb_downsampled_sigmoid_on_cv, open(work_dir+'/models/pre_anat_intra/'+'model_' +outcome + '_cv_cb_downsampled_isotonic_calibration_on_cv.sav', 'wb'))


In [29]:
# Variable Importance:

feature_imp_df = cv_cb_downsampled.get_feature_importance(prettified = True)
feature_imp_df_sort = feature_imp_df.sort_values(by='Importances', ascending=False, inplace=False, kind='quicksort', na_position='last')
feature_imp_df_sort.to_csv(load_dir+'/'+'feature_imp_df_sort_'+outcome+'.csv', index=False)

decouple_var_imp_sort = decouple_var_imp(feature_imp_df_sort)
decouple_var_imp_sort.to_csv(load_dir+'/'+'decouple_var_imp_sort_'+outcome+'.csv', index=False)
decouple_var_imp_sort.iloc[:10,:]

/home/idies/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Feature Id,total_importance,min_max_scale_importance
1,status,7.035540,1.000000
6,age,5.289334,0.751087
67,ppef,5.002661,0.710223
4,rfhemoglobin,4.753362,0.674687
25,perfustm,4.257457,0.603998
14,totalbumin,4.170297,0.591574
37,pasys,3.884349,0.550813
28,bmi,3.798635,0.538595
24,wbc,3.617348,0.512754
5,creatlst,3.590111,0.508871


In [ ]:
# Bootstrap testset:
bs_testset_performance_df = pd.DataFrame()
bs_testset_performance_isotonic_df = pd.DataFrame()
bs_testset_performance_sigmoid_df = pd.DataFrame()
for i in range(100):
    sample_indices = np.random.randint(len(X_test), size=len(X_test))
    bs_y_test = y_test[sample_indices]
    bs_X_test = X_test[sample_indices,:]
    
    bs_y_test_predict_proba_cb_downsampled = cv_cb_downsampled.predict_proba(bs_X_test)[:, 1]
    bs_performance_metrics = performance_wo_figure(bs_y_test, bs_y_test_predict_proba_cb_downsampled, thres = 0.5)
    bs_testset_performance_df = bs_testset_performance_df.append(bs_performance_metrics, ignore_index=True)

    
    bs_y_test_predict_proba_cb_downsampled_isotonic = cv_cb_downsampled_isotonic_on_cv.predict_proba(bs_X_test)[:, 1]
    bs_performance_metrics_isotonic = performance_wo_figure(bs_y_test, bs_y_test_predict_proba_cb_downsampled_isotonic, thres = 0.5)
    bs_testset_performance_isotonic_df = bs_testset_performance_isotonic_df.append(bs_performance_metrics_isotonic, ignore_index=True)
    
    bs_y_test_predict_proba_cb_downsampled_sigmoid = cv_cb_downsampled_sigmoid_on_cv.predict_proba(bs_X_test)[:, 1]
    bs_performance_metrics_sigmoid = performance_wo_figure(bs_y_test, bs_y_test_predict_proba_cb_downsampled_sigmoid, thres = 0.5)
    bs_testset_performance_sigmoid_df = bs_testset_performance_sigmoid_df.append(bs_performance_metrics_sigmoid, ignore_index=True)

In [ ]:
bs_testset_performance_df.to_csv(load_dir+'/'+'bs_testset_performance_df_' + outcome +'.csv', index=False)
bs_testset_performance_isotonic_df.to_csv(load_dir+'/'+'bs_testset_performance_isotonic_df_' + outcome +'.csv', index=False)
bs_testset_performance_sigmoid_df.to_csv(load_dir+'/'+'bs_testset_performance_sigmoid_df_' + outcome +'.csv', index=False)

In [ ]:
bs_testset_performance_df.describe(percentiles = [.025, .5, .975]).round(decimals = 3)

In [ ]:
# print out performance:
print_bs_testset_performance = pd.DataFrame(bs_testset_performance_df[['AUROC', 'accuracy', 'ARC', 'Brier', 'F1','precision','recall', 'sens', 'spec']].apply(lambda x: report_metric(x), axis = 0)).transpose()
print_bs_testset_performance.to_csv(load_dir+'/'+'print_bs_testset_performance_' + outcome +'.csv', index=False)
print_bs_testset_performance

In [ ]:
# print out performance:
print_bs_testset_performance_isotonic = pd.DataFrame(bs_testset_performance_isotonic_df[['AUROC', 'accuracy', 'ARC', 'Brier', 'F1','precision','recall', 'sens', 'spec']].apply(lambda x: report_metric(x), axis = 0)).transpose()
print_bs_testset_performance_isotonic.to_csv(load_dir+'/'+'print_bs_testset_performance_isotonic_' + outcome +'.csv', index=False)
print_bs_testset_performance_isotonic

In [ ]:
# print out performance:
print_bs_testset_performance_sigmoid = pd.DataFrame(bs_testset_performance_sigmoid_df[['AUROC', 'accuracy', 'ARC', 'Brier', 'F1','precision','recall', 'sens', 'spec']].apply(lambda x: report_metric(x), axis = 0)).transpose()
print_bs_testset_performance_sigmoid.to_csv(load_dir+'/'+'print_bs_testset_performance_isotonic_' + outcome +'.csv', index=False)
print_bs_testset_performance_sigmoid

In [ ]:
# Plot performance figures:
performance(bs_y_test, bs_y_test_predict_proba_cb_downsampled, thres = 0.5)

In [ ]:
# import matplotlib as plt
# plt.rcParams["figure.figsize"] = (12,7)
# ax = feature_imp_df_sort.plot('Feature', 'Score', kind='bar', color='c')
# ax.set_title("Feature Importance using Permutation method", fontsize = 14)
# ax.set_xlabel("features")


In [ ]:
# # Train and calibrate model on entirety of the training set:

# X_train_downsampled = df_downsampled.drop(['label'],axis =1)
# y_train_downsampled = df_downsampled['label'].ravel()
# from catboost import CatBoostClassifier
# cb_downsampled =  CatBoostClassifier(random_state = 1)
# cb_downsampled.fit(X_train_downsampled, y_train_downsampled, cat_features = np.array(get_cat_features(X_train_downsampled, 5)), verbose = False)
# pickle.dump(cb_downsampled, open('model_' +outcome + '_cb_downsampled.sav', 'wb'))

# # Create a corrected classifier. - Isotonic calibration
# cb_downsampled_iso = sklearn.calibration.CalibratedClassifierCV(cb_downsampled, cv=10, method='isotonic')
# cb_downsampled_iso.fit(X_train_downsampled, y_train_downsampled.ravel())

# pickle.dump(cb_downsampled_iso, open('model_' +outcome + '_cb_downsampled_iso.sav', 'wb'))

# cb_downsampled_sigmoid = sklearn.calibration.CalibratedClassifierCV(cb_downsampled, cv=10, method='sigmoid')
# cb_downsampled_sigmoid.fit(X_train_downsampled, y_train_downsampled.ravel())
# pickle.dump(cb_downsampled_sigmoid, open('model_' +outcome + '_cb_downsampled_sigmoid.sav', 'wb'))

# # loaded_model = pickle.load(open('model_' +outcome + '_cb_downsampled_iso.sav', 'rb'))
# # y_test_predict_proba_loaded_model = loaded_model.predict_proba(X_test)[:, 1]

# # # Bootstrap testset:
# # bs_testset_performance_df = pd.DataFrame()
# # for i in range(10):
# #     sample_indices = np.random.randint(len(X_test), size=len(X_test))
# #     bs_y_test = y_test[sample_indices]
# #     bs_X_test = X_test[sample_indices,:]
# #     bs_y_test_predict_proba_cb_downsampled_iso = cb_downsampled_iso.predict_proba(bs_X_test)[:, 1]
# #     bs_performance_metrics = performance(bs_y_test, bs_y_test_predict_proba_cb_downsampled_iso, thres = 0.5)
# #     bs_testset_performance_df = bs_testset_performance_df.append(bs_performance_metrics, ignore_index=True)


In [ ]:
# # Bootstrap testset and spit prediction using calibrated model:
# import pickle
# import sklearn
# iso_model = pickle.load(open('model_' +outcome + '_cb_downsampled_iso.sav', 'rb'))

# bs_testset_pred_df = pd.DataFrame(list(zip(data_full['concatid'][test_index], test_index)),
#                                 columns=['concatid', 'index'])
# bs_testset_performance_iso_df = pd.DataFrame()
# for i in range(10):
#     sample_indices = np.random.randint(len(X_test), size=len(X_test))
#     bs_y_test = y_test[sample_indices]
#     bs_X_test = X_test[sample_indices,:]
#     bs_y_test_predict_proba_cb_downsampled_iso = iso_model.predict_proba(bs_X_test)[:, 1]
#     bs_performance_metrics = performance_wo_figure(bs_y_test, bs_y_test_predict_proba_cb_downsampled_iso, thres = 0.5)
#     bs_testset_performance_iso_df = bs_testset_performance_iso_df.append(bs_performance_metrics, ignore_index=True)


In [ ]:
# bs_testset_performance_iso_df

In [ ]:
# iso_model = pickle.load(open('model_' +outcome + '_cb_downsampled_iso.sav', 'rb'))
# y_test_predict_proba_cb_downsampled_iso = iso_model.predict_proba(X_test)[:, 1]
# testset_iso_pred_df = pd.DataFrame(list(zip(y_test, y_test_predict_proba_cb_downsampled_iso)),
#                                 columns=['label', 'pred'])
# testset_iso_pred_df.to_csv(load_dir+'/'+'pred_prob_testset_iso_' + outcome +'.csv', index=False)

In [ ]:
# before_iso_model = pickle.load(open('model_' +outcome + '_cv_cb_downsampled.sav', 'rb'))
# y_test_predict_proba_cb_downsampled = before_iso_model.predict_proba(X_test)[:, 1]
# testset_pred_df = pd.DataFrame(list(zip(y_test, y_test_predict_proba_cb_downsampled)),
#                                 columns=['label', 'pred'])
# testset_pred_df.to_csv(load_dir+'/'+'pred_prob_testset_before_iso_' + outcome +'.csv', index=False)

In [ ]:
# import pickle
# from sklearn import calibration
# cv_cb_downsampled = pickle.load(open('model_' +outcome + '_cv_cb_downsampled.sav', 'rb'))
# cv_cb_downsampled_sigmoid = sklearn.calibration.CalibratedClassifierCV(cv_cb_downsampled, cv=10, method='sigmoid')
# cv_cb_downsampled_sigmoid.fit(X_train, y_train.ravel())
# pickle.dump(cv_cb_downsampled_sigmoid, open('model_' +outcome + '_cv_cb_downsampled_sigmoid.sav', 'wb'))


In [ ]:
# cv_sigmoid_model = pickle.load(open('model_' +outcome + '_cv_cb_downsampled_sigmoid.sav', 'rb'))
# y_test_predict_proba_cb_downsampled = cv_sigmoid_model.predict_proba(X_test)[:, 1]
# testset_pred_df = pd.DataFrame(list(zip(y_test, y_test_predict_proba_cb_downsampled)),
#                                 columns=['label', 'pred'])
# testset_pred_df.to_csv(load_dir+'/'+'pred_prob_testset_sigmoid_' + outcome +'.csv', index=False)

In [ ]:
# import pickle
# from sklearn import calibration
# cv_cb_downsampled = pickle.load(open('model_' +outcome + '_cv_cb_downsampled.sav', 'rb'))
# cv_cb_downsampled_isotonic = sklearn.calibration.CalibratedClassifierCV(cv_cb_downsampled, cv=10, method='isotonic')
# cv_cb_downsampled_isotonic.fit(X_train, y_train.ravel())
# pickle.dump(cv_cb_downsampled_isotonic, open('model_' +outcome + '_cv_cb_downsampled_isotonic.sav', 'wb'))
# cv_isotonic_model = pickle.load(open('model_' +outcome + '_cv_cb_downsampled_isotonic.sav', 'rb'))
# y_test_predict_proba_cb_downsampled = cv_isotonic_model.predict_proba(X_test)[:, 1]
# testset_pred_df = pd.DataFrame(list(zip(y_test, y_test_predict_proba_cb_downsampled)),
#                                 columns=['label', 'pred'])
# testset_pred_df.to_csv(load_dir+'/'+'pred_prob_testset_isotonic_' + outcome +'.csv', index=False)


In [ ]:
# import pickle
# from sklearn import calibration
# cv_cb_downsampled = pickle.load(open('model_' +outcome + '_cv_cb_downsampled.sav', 'rb'))
# cv_cb_downsampled_isotonic_on_db = sklearn.calibration.CalibratedClassifierCV(cv_cb_downsampled, cv=10, method='isotonic')
# X_train_downsampled = df_downsampled.drop(['label'],axis =1)
# y_train_downsampled = df_downsampled['label'].ravel()
# cv_cb_downsampled_isotonic_on_db.fit(X_train_downsampled, y_train_downsampled.ravel())
# pickle.dump(cv_cb_downsampled_isotonic_on_db, open('model_' +outcome + '_cv_cb_downsampled_isotonic_on_db.sav', 'wb'))
# cv_isotonic_model_on_db = pickle.load(open('model_' +outcome + '_cv_cb_downsampled_isotonic_on_db.sav', 'rb'))
# y_test_predict_proba_cb_downsampled = cv_isotonic_model_on_db.predict_proba(X_test)[:, 1]
# testset_pred_df = pd.DataFrame(list(zip(y_test, y_test_predict_proba_cb_downsampled)),
#                                 columns=['label', 'pred'])
# testset_pred_df.to_csv(load_dir+'/'+'pred_prob_testset_isotonic_on_db_' + outcome +'.csv', index=False)


In [ ]:
# import pickle
# from sklearn import calibration
# cv_cb_downsampled = pickle.load(open('model_' +outcome + '_cv_cb_downsampled.sav', 'rb'))
# cv_cb_downsampled_isotonic_on_cv = sklearn.calibration.CalibratedClassifierCV(cv_cb_downsampled, cv=10, method='isotonic')
# cv_cb_downsampled_isotonic_on_cv.fit(cv_X_train, cv_y_train.ravel())
# pickle.dump(cv_cb_downsampled_isotonic_on_cv, open('model_' +outcome + '_cv_cb_downsampled_isotonic_on_cv.sav', 'wb'))
# cv_isotonic_model_on_cv = pickle.load(open('model_' +outcome + '_cv_cb_downsampled_isotonic_on_cv.sav', 'rb'))
# y_test_predict_proba_cb_downsampled = cv_isotonic_model_on_cv.predict_proba(X_test)[:, 1]
# testset_pred_df = pd.DataFrame(list(zip(y_test, y_test_predict_proba_cb_downsampled)),
#                                 columns=['label', 'pred'])
# testset_pred_df.to_csv(load_dir+'/'+'pred_prob_testset_isotonic_on_cv_' + outcome +'.csv', index=False)


### Holdout testset feature space and sts_pred_prob filter:


In [ ]:

# testset_df = data_full.iloc[test_index,:]
# testset_df.to_csv(load_dir+'/'+'testset_feature_df_' + outcome +'.csv', index=True)
# # load sts pred prob and filter only patients in the holdout test set:
# load_dir = '/home/idies/workspace/Storage/hnguye78/persistent'+'/csv_files'
# sts_pred_prob_df = pd.read_csv(load_dir+'/sts_pred_prob.csv')
# sts_pred_prob_df['concatid'] = sts_pred_prob_df['patid']+sts_pred_prob_df['recordid']
# sts_pred_prob_df_rm_dups = sts_pred_prob_df.drop_duplicates(subset='concatid', keep="last")
# sts_pred_prob_testset = sts_pred_prob_df_rm_dups.loc[sts_pred_prob_df_rm_dups['concatid'].isin(testset_df['concatid']),['concatid', 'predvent']]

# sts_pred_prob_testset.to_csv(load_dir+'/'+'sts_pred_prob_testset_' + outcome +'.csv', index=False)
# # # check to make sure the concatid in sts_pred_prob and in the testset are the same: 
# print(sorted(list(sts_pred_prob_testset['concatid'])) == sorted(list(testset_df['concatid'])))
# # # True

### Predicted Probability of Patients in Non-STS Set:

In [ ]:
# load_dir = '/home/idies/workspace/Storage/hnguye78/persistent'+'/csv_files'
# feature_space_non_sts_patients = pd.read_csv(load_dir+'/non_sts_patients_feature_space.csv')
# X_non_sts = feature_space_non_sts_patients.drop(['label','Unnamed: 0', 'concatid'], axis = 1).values
# import pickle
# import sklearn
# loaded_model = pickle.load(open('model_' +outcome + '_cv_cb_downsampled.sav', 'rb'))
# y_non_sts_predict_proba = loaded_model.predict_proba(X_non_sts)[:, 1]

# prob_risk_non_sts_df = pd.DataFrame(list(zip(feature_space_non_sts_patients['concatid'], y_non_sts_predict_proba)),
#                                 columns=['concatid', 'predicted_prob'])
# prob_risk_non_sts_df.to_csv(load_dir+'/'+'pred_prob_non_sts_patients_df_' + outcome +'.csv', index=False)
# # # check to see that the non_sts set does not overlap with the testset concatid 
# # set((prob_risk_non_sts_df['concatid'])).intersection((set(testset_df['concatid'])))
# # # no overlap